In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Kidney_Papillary_Cell_Carcinoma/GSE63003'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Renal cells Carcinoma (RCC) Genome variation profiling by genome tiling array"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: Papillary renal cell carcinoma (pRCC)', 'cell type: unclassified renal cell carcinoma (uRCC)'], 1: ['rcc type: type 1', 'rcc type: type 2', 'rcc type: unclassified']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

is_gene_available = True  # Since this is a genome variation profiling dataset, it is likely to contain gene expression data.

# Check availability of variables
sample_characteristics = {
    0: ['cell type: Papillary renal cell carcinoma (pRCC)', 'cell type: unclassified renal cell carcinoma (uRCC)'], 
    1: ['rcc type: type 1', 'rcc type: type 2', 'rcc type: unclassified']
}

trait_row = 0
age_row = None  # age information not explicitly available
gender_row = None  # gender information not explicitly available

# Define conversion functions
def convert_trait(value):
    val = value.split(': ')[-1].strip().lower()
    if val == 'papillary renal cell carcinoma (prcc)':
        return 1
    elif val == 'unclassified renal cell carcinoma (urcc)':
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(': ')[-1].strip().lower()
    if val == 'male':
        return 1
    elif val == 'female':
        return 0
    else:
        return None

save_cohort_info('GSE63003', './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Kidney_Papillary_Cell_Carcinoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/trait_data/GSE63003.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1537868': [1], 'GSM1537869': [1], 'GSM1537870': [1], 'GSM1537871': [1], 'GSM1537872': [1], 'GSM1537873': [1], 'GSM1537874': [1], 'GSM1537875': [0], 'GSM1537876': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['532', '532', '532', '532', '532'], 'ROW': ['340', '338', '336', '334', '332'], 'PROBE NAME': ['HsCGHBrightCorner', 'DarkCorner2', 'DarkCorner2', 'DarkCorner2', 'DarkCorner2'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'CONTROL', 'CONTROL']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and assign the resulting gene expression dataframe to "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE63003.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Kidney_Papillary_Cell_Carcinoma')

# 4. Save the cohort information.
json_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/cohort_info.json'
save_cohort_info('GSE63003', json_path, True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Kidney_Papillary_Cell_Carcinoma/GSE63003.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Kidney_Papillary_Cell_Carcinoma', the least common label is '0.0' with 2 occurrences. This represents 22.22% of the dataset.
The distribution of the feature 'Kidney_Papillary_Cell_Carcinoma' in this dataset is fine.

